# Diesel Consumption Report Automation Algomodule Documentation

Created by: Chan Xing Wei

Created On: 17/03/2022

Last Updated: 01/04/2022 08:43 AM

## Changelog
### Version 2.1 [01/04/2022 08:43 AM]
* support for real XLS importing if XML importing failed
* Ask user to move the source files into the Data backup folder and delete all source files after complete importing without unhandled failure
* Ask if the user wants to go back to main menu after completing import all data
* record any unhandled error down the error log, name of skipped file and timestamp for post-program debugging

### Version 2.0 [30/03/2022 01:32 AM]
This version has been updated with the following changes
* Minor fine tuning
* Menu to operate two different operations:
    * Run the program
    * Configuration of parameters (two layer: simple & advanced)
* All settings are contained into a dictionary for easy save and load configuration
* Splitted the menu and config codes from the algorithm codes
* skip any file that raise unhandled error

### Version 1.1 [22/03/2022 01:44 PM]
* Added progress bar for lines writen
* Enhanced results printing (showing successful imported files, empty files and skipped files)
* Edited docstrings

### Version 1.0 [22/03/2022 12:00 AM]
The first version of this automation script, includes all basic and necessary features to make this script usable

## Abstract
This document contains everything about this automation algorithm module.

This module should include all classes and algorithmic functions that are involved in the entire automation program.

The main reason of separating these algorithm out as a separate module is to provide cleaner code, ultimately providing higher scalability to the program for other automation in the future (if any).

## Outline
* Changelog
* Abstract
* Workflow
* Imports
* Variables
* Classes
    * DieselReportMainException
    * DirEmptyException
    * FileNotExistException
    * FileEmptyException
    * Etime
* Functions
    * XML to CSV
    * XLS to CSV
    * Insert Import Date
    * Vehicle Processing
    * Truck Type Processing
    * Date Processing
    * Move Files
    * Ask for Quit
    * Make Log
    * Automate Import Data

## Workflow

![Workflow](https://drive.google.com/uc?id=10XK9ExzokuV_CgP7rYiRjSjMSqfOOqn8&authuser=xingwei.chan%40gotruck.com&usp=drive_fs)

## Imports

In [ ]:
import sys
import os
import time
import datetime
import pandas as pd
from openpyxl import Workbook
from openpyxl import load_workbook
from openpyxl.utils.dataframe import dataframe_to_rows
import xlrd

## Variables

|***Variables***|Description|Authority|
|-|-|-|
|***destpath***|The folder directory of where the main diesel report excel file is located, it can be changed but user has to make sure the integrity of the necessary files in the directory|Basic|
|***importpath***|The directory folder name of where the source files to be imported are located|Basic|
|***backuppath***|The directory folder name of where the files stored as backup after successful importing|Basic|
|***dieselyear***|This can be change so to enable this script to be used in any year|Basic|
|***mainexcel***|The destination excel file name where all the final data should be stored|Advanced|
|***mainexcelsheet***|The destination excel sheet within the excel file specified in ***mainexcel***|Advanced|
|***mixerlist***|A default list containing the first letter of truck number for concrete mixer truck type determination|Advanced|
|***pickuplist***|A default list containing the first letter of truck number for pick up truck type determination|Advanced|
|***tipperlist***|A default list containing the first letter of truck number for tipper truck type determination|Advanced|
|***wheelloaderlist***|A default list containing the first letter of truck number for wheel loader truck type determination|Advanced|
|***dispsplit1***|A parameter to control the completion progress bar split length (i.e. 50 means the progress bar will be splitted with a newline and display a multiplier of 50 for every 50 files imported)|Advanced|
|***dispsplit2***|A parameter to control the completion progress bar split length (i.e. 50 means the progress bar will be splitted with a newline and display a multiplier of 50 for every 50 files imported)|Advanced|
|***benchmarking***|A parameter to determine is the console required to print benchmarking info (granular elapsed time)|Advanced|
|***tmpcsv***|A temporary string stored in csv format|Internal Use|
|***importsuccess***|A counter counting how many successful importation|Internal Use|
|***importempty***|A counter counting how many empty files leading to skipped importation|Internal Use|
|***imported***|A list containing all successful imported filename|Internal Use|
|***emptyfile***|A list containing all failed imported filename due to file is empty|Internal Use|
|***nonxlscount***|A counter counting how many non XLS files in the import source folder|Internal Use|
|***nonxls***|A list containing all skipped filename due to the file extension is not .xls|Internal Use|
|***skippedforerrorcount***|A counter counting how many files that raised unexpected error in the import source folder|Internal Use|
|***skippedforerror***|A list containing all skipped filename that raised unexpected error|Internal Use|
|***bytewriten***|The size of content that had been successfully writen to the destination excel file, in bytes|Internal Use|
|***linewriten***|A counter for how many lines that had been successfully writen to the destination excel file|Internal Use|

In [ ]:
tmpcsv = ""

## Classes

### DieselReportMainException
This is the main exception exclusive to this program ONLY, it is directly inherited from BaseException and SHOULD NOT be directly used

In [ ]:
class DieselReportMainException(BaseException):
    '''
    Main exception exclusive to this program ONLY.
    Directly inherited from BaseException.
    SHOULD NOT be directly used.
    '''
    pass

### DirEmptyException
This exception will be raised when the directory passed as argument does not contain any readable file

In [ ]:
class DirEmptyException(DieselReportMainException):
    '''
    This exception will be raised when the directory passed as argument
    does not contain any readable file.
    '''
    def __init__(self,path):
        self.path = path
        self.msg = f"{DirEmptyException.__name__}: No file found in {self.path}"

### FileNotExistException
This exception will be raised when the file passed as argument does not exist

In [ ]:
class FileNotExistException(DieselReportMainException):
    '''This exception will be raised when the file passed as argument does not exist.'''
    def __init__(self,filename):
        self.msg = f"{FileNotExistException.__name__}: {filename} does not exist!"

### FileEmptyException
This exception will be raised when the file passed as argument is empty

In [ ]:
class FileEmptyException(DieselReportMainException):
    '''This exception will be raised when the file passed as argument is empty.'''
    def __init__(self,filename):
        self.msg = f"{FileEmptyException.__name__}: {filename} is empty!"

### Etime

The object created from this class should invoke an elapsed time function and save it as its private properties

* There is a method which takes in one argument in Etime and will return the difference in time between the current and the target Etime object. The time will be automatically converted to human readable unit

Packages used:
* time
    * time.perf_counter()
    * time.process_time()

In [ ]:
class Etime():
    '''
    The object created from this class should invoke an time.perf_counter()
    function and save it as its private properties.
    '''
    def __init__(self):
        self.t = time.perf_counter()

    def __repr__(self):
        return str(self.t)

    def difftime(self,etime_obj):
        '''
        Takes in Etime object and return the difference in time (t2-t1)
        between the current and the target Etime object.
        Results will be in absolute value.
        The time will be automatically converted to human readable unit.
        '''
        _diff = abs(etime_obj.t - self.t)

        if _diff >= 86400: return f"{round((_diff / 86400),1)}"+" days"
        elif _diff >= 3600: return f"{round((_diff / 3600),1)}"+" hours"
        elif _diff >= 60: return f"{round((_diff / 60),1)}"+" mins"
        elif _diff >= 1: return f"{round((_diff),1)}"+" seconds"
        else: return f"{round(_diff * 1000)}"+" ms"

## Functions

### XML to CSV
This funciton will do the following:
* Read line by line from xml file(s) from a source file passed as the argument of this function
* Ignore irrelevant lines
* Reformat the relevant lines into a "a,b,c,d,..." line format
* Write the reformatted lines into a temporary CSV formatted string


Packages used:
* os
    * os.strerror()
* errno
    * errno.errorcode [Dict]

In [ ]:
def xml2csv(filename,config):
    '''
    Takes in a filename and extract necessary information from the XML file.
    
    "config" is a dictionary containing the master configuration settings and
    it should be passed down all the way from the main program.

    It should return "success" or "empty" for counting of sucessful or failure imports.
    '''
    _rowtgt = "<Row"
    _rowfound = False
    _datatgt = "<Data"
    _datafound = False
    _yeartgt = "/"+config['dieselyear']
    _yearfound = False
    _tmpline = ""
    global tmpcsv

    if not tmpcsv: tmpcsv += "Date / Time,Station,PAN 1,Customer,Vehicle,Article,Quantity"
    try:
        with open(config['importpath'] + filename,'r',encoding='utf-8') as fl:
            r = fl.readline()
            if not r:
                e1 = FileEmptyException(filename)
                makelog(e1,xml2csv.__name__,True)
            else:
                while r:
        ##            print(1,end='')
                    if r[0:4] == _rowtgt:
        ##                print(2,end='')
                        _rowfound = True
                        while _rowfound:
                            if r[0:5] == "</Row":
        ##                        print(2,end='')
                                if "diesel" in _tmpline.lower() or "adblue" in _tmpline.lower():
                                    tmpcsv += "\n"
                                    tmpcsv += _tmpline
                                _tmpline = ""
                                _yearfound = False
                                _rowfound = False
                                r = fl.readline()
                                break
                            elif r[0:5] == _datatgt:
        ##                        print(3,end='')
                                if _yeartgt in r: _yearfound = True
                                if _rowfound and _yearfound:
        ##                            print(4,end='')
                                    _tmpchr = ""
                                    _takein = 0
                                    for char in r.replace("\n",""):
                                        if char == "<":
                                            _takein = 0
                                            continue
                                        elif _takein == 1: _tmpchr += char
                                        elif char != ">": continue
                                        elif char == ">": _takein += 1
                                    else:
                                        if _tmpchr: _tmpchr += ","
                                        _tmpline += _tmpchr
                                        r = fl.readline()
                                        continue
                                else:
                                    r = fl.readline()
                                    continue
                            else:
                                r = fl.readline()
                                continue
                    else:
                        _rowfound = False
                        r = fl.readline()
                        continue
            fl.close()
    except IOError as e:
        em = f"Error accessing '{filename}' in {config['importpath']}"
        makelog(e,xml2csv.__name__,em,True)
        print(os.strerror(e.errno))
        askquit()
    except UnicodeError as euni:
        em = f"Error encoding/decoding the '{filename}'"
        makelog(euni,xml2csv.__name__,em,True)
        raise        
    except Exception as e2:
        em = f"Error reading data from '{filename}'"
        makelog(e2,xml2csv.__name__,em)
        print("X",end='')
        return "error"
    else:
        if "e1" in locals():
            print("X",end='')
            return "empty"            
        else:
            print("|",end='')
            return "success"

### XLS to CSV
This funciton will do the following:
* Read XLS file with xldr
* reformat the content into row arrays
* ignoring the irrelevant lines ("Diesel" or "AdBlue" not found)
* Write the reformatted lines into a temporary CSV formatted string


Packages used:
* os
    * os.strerror()
* errno
    * errno.errorcode [Dict]
* xlrd
    * xlrd.open_workbook()
    * xlrd.Book
        * Book.release_resources()
    * xlrd.Sheet
        * Sheet.col_slice()
        * Sheet.row_values(rowx)

In [ ]:
def xls2csv(filename,config):
    '''
    Takes in a filename and extract necessary information from the XLS file.
    
    "config" is a dictionary containing the master configuration settings and
    it should be passed down all the way from the main program.
    '''
    global tmpcsv

    if not tmpcsv: tmpcsv += "Date / Time,Station,PAN 1,Customer,Vehicle,Article,Quantity"
    try:
        book = xlrd.open_workbook(config['importpath']+filename)
        s = book[0]
        rownum = len(s.col_slice(0))

        for n in range(rownum):
            _tmpline = ""
            d = False
            currow = s.row_values(n)   
            
            for cellval in currow:
                if str(cellval).lower() == 'diesel' or str(cellval).lower() == 'adblue': d = True
                if cellval:
                    _tmpline += str(cellval) + ','
            
            if d: tmpcsv += '\n' + _tmpline
        book.release_resources()
        del book
    except IOError as e:
        em = f"Error accessing '{filename}' in {config['importpath']}"
        makelog(e,xml2csv.__name__,em,True)
        print(os.strerror(e.errno))
        askquit()
    except Exception as e2:
        em = f"Error reading data from '{filename}'"
        makelog(e2,xml2csv.__name__,em)
        print("X",end='')
        return "error"
        raise
    else:
        if "e1" in locals():
            print("X",end='')
            return "empty"            
        else:
            print("|",end='')
            return "success"

### Insert Import Date
This function accepts ONLY dataframe as argument and insert a new column "Imported" containing the imported date (the day of running this program using datetime.date.today()), and return the elapsed time from start to end of invocation of this function for benchmarking

In [ ]:
def ins_import_date(dframe):
    '''
    Takes in a dataframe and insert a new column "Imported".

    "Imported" contains the imported date (the day of running this program
    using datetime.date.today()).

    It should return the elapsed time from end to end of running this function
    for benchmarking purpose.
    '''
    t0 = Etime()
    dframe.insert(len(dframe.columns),"Imported",\
    [time.strftime("%d/%m/%Y",time.strptime(str(datetime.date.today()),"%Y-%m-%d")) \
    for x in range(len(dframe.index))])
    t1 = Etime()

    return t1.difftime(t0)

### Vehicle Processing
This function accepts ONLY dataframe as argument and insert a new column "Truck No" containing a reformatted truck number for excel calculation purposes, and return the elapsed time from start to end of invocation of this function for benchmarking

Logic:
1. If truck number contains "(" and ")": set truck number to string within "(" and ")"
2. Else: set truck number to 0

In [ ]:
def vehicle_process(dframe):
    '''
    Takes in a dataframe and insert a new column "Truck No".

    "Truck No" contains a reformatted truck number for excel
    calculation purposes.

    It should return the elapsed time from end to end of running this function
    for benchmarking purpose.
    '''
    writetmp = False
    tmpchr = ""
    moddf = []
    s1 = []

    t0 = Etime()
    for x in dframe["Vehicle"]:
        moddf.append(x.replace(" ",""))
        if '(' not in x and ')' not in x:
            s1.append(0)
        else:
            for char in x:
                if char == ')': writetmp = False
                elif writetmp and char != ')': tmpchr += char
                elif char == '(': writetmp = True
                else: continue
            s1.append(tmpchr)
            tmpchr = ""

    dframe["Vehicle"] = moddf
    dframe.insert(len(dframe.columns),"Truck No",s1)
    t1 = Etime()

    return t1.difftime(t0)

### Truck Type Processing
This function accepts ONLY dataframe as argument and insert a new column "Truck Type" containing the defined truck type for excel calculation purposes, and return the elapsed time from start to end of invocation of this function for benchmarking

Logic:
1. If location == "United Cement (Jurong Port)": set truck type to "Cement Tanker" (Hardcoded)
2. If first letter of truck number is in mixerlist: set truck type to "Mixer Truck"
3. If first letter of truck number is in pickuplist: set truck type to "Pick Up"
4. If first letter of truck number is in tipperlist: set truck type to "Tipper Truck"
5. If first letter of truck number is in wheelloaderlist: set truck type to "Wheel Loader"


In [ ]:
def truck_type_process(dframe,config):
    '''
    Takes in a dataframe and insert a new column "Truck Type".

    "Truck Type" contains the defined truck type for excel calculation purposes.

    It should return the elapsed time from end to end of running this function
    for benchmarking purpose.
    '''
    lists = [dframe["Station"],dframe["Vehicle"]]
    s2 = []
    t0 = Etime()

    for a,b in zip(lists[0],lists[1]):
        if a == "United Cement (Jurong Port)": s2.append("Cement Tanker")
        elif b[0] in config['mixerlist']: s2.append("Mixer Truck")
        elif b[0] in config['pickuplist']: s2.append("Pick Up")
        elif b[0] in config['tipperlist']: s2.append("Tipper Truck")
        elif b[0] in config['wheelloaderlist']: s2.append("Wheel Loader")

    dframe.insert(len(dframe.columns),"Truck Type",s2)
    t1 = Etime()

    return t1.difftime(t0)

### Date Processing
This function accepts ONLY dataframe as argument and insert two new columns "Date" and "Month"  for excel calculation purposes, and return the elapsed time from start to end of invocation of this function for benchmarking

In [ ]:
def date_process(dframe):
    '''
    Takes in a dataframe and insert two new columns "Date" and "Month" .

    Mainly for excel calculation purposes.

    It should return the elapsed time from end to end of running this function
    for benchmarking purpose.
    '''
    dtlist = []
    s3 = []
    s4 = []
    t0 = Etime()

    for d in dframe["Date / Time"]:
        d2 = d
        if "/" in d[0:2]: d2 = "0"+d2
        if d2[3] != "0":
            l = list(d2)
            l.insert(3,"0")
            d2 = "".join(l)
        dtlist.append(d2)

    for d in dtlist:
        d2 = [x.strip() for x in d.split()][0]
        t_obj = time.strptime(d2,"%d/%m/%Y")
        s3.append(time.strftime("%d/%m/%Y",t_obj))
        s4.append(time.strftime("%b",t_obj))

    dframe.insert(len(dframe.columns),"Date",s3)
    dframe.insert(len(dframe.columns),"Month",s4)
    t1 = Etime()

    return t1.difftime(t0)

### Move Files
This function ask if the user wants move all successful imported source files into a backup folder.

The user is asked to input a folder name if he selected to move files, all the files wil be moved to under this folder under the backup directory.

In [ ]:
def movefiles(lst,config):
    '''
    Takes in a list of filenames (successful imported file list) and the global
    config dictionary, ask the user if he wants to move all successful imported
    files into a backup folder in backup directory.
    '''
    movecount = 0
    moved = []
    movefailed = 0
    movefaileds = []
    
    print("Data importing completed. Do you want to move the successful imported files to backup folder? (Y/N)")
    print(f"Backup folder: {config['backuppath']}")
    try:
        ans = input("Action: ")
        assert ans.lower() in ['y','n']
    except AssertionError:
        print("Please input 'Y' or 'N' only!\n")
        movefiles(config)
    else:
        if ans.lower() == 'y':
            folder = input("\nPlease enter the folder name to contain all the files of this current batch: ")
            folder += '\\'
            
            try:
                os.mkdir(config['backuppath']+folder)
                for file in lst:
                    if file.lower() == 'desktop.ini': continue
                    elif file.lower() != 'desktop.ini':
                        
                        try:
                            os.replace(config['importpath']+file,config['backuppath']+folder+file)
                        except IOError as eio:
                            movefailed += 1
                            movefaileds.append(file)
                            em = f"Error moving file '{file}' to {config['backuppath']+folder}"
                            makelog(eio,movefiles.__name__,em,True)
                            print(f"Failed to move file to {config['backuppath']+folder}.")
                            print("Files that failed to move:")
                            [print(x,end=',') for x in movefaileds]
                            sys.exit()
                        except Exception as e:
                            movefailed += 1
                            movefaileds.append(file)
                            print("X",end='')
                            em = f"Error moving file '{file}' to {config['backuppath']+folder}"
                            makelog(e,movefiles.__name__,em)
                            continue
                        else:
                            movecount += 1
                            moved.append(file)
                            print("|",end='')
                            if movecount == config['dispsplit1']: print(f" {movecount}\n")
                            
                else:
                    if movecount != config['dispsplit1']: print(f" {movecount}\n")
                    print(f"{movecount} files successfully moved to {config['backuppath']+folder}")
                    if movefaileds:
                        print("Files that failed to move:")
                        [print(x,end=',') for x in movefaileds]
            except IOError as eio:
                em = "Error creating folder '{folder}' in {config['backuppath']}"
                makelog(eio,movefiles.__name__,em,True)
                print(f"Failed to create the folder {folder} in {config['backuppath']}.")
                print("Moving operation will be stopped.")
                print("Files that failed to move:")
                [print(x,end=',') for x in movefaileds]
                sys.exit()
            except Exception as e:
                em = "Error creating folder '{folder}' in {config['backuppath']}"
                makelog(e,movefiles.__name__,em)
                print(f"Failed to create the folder {folder} in {config['backuppath']}.")
                print("Moving operation will be stopped.")
                print("Files that failed to move:")
                [print(x,end=',') for x in movefaileds]
                
        else:
            print("Files will not be moved, please remove the files from the import source folder to avoid duplicate importing")

### Ask for Quit
This function ask if the user wants to quit this program

In [ ]:
def askquit(newline=False):
    '''This function ask if the user wants to quit this program.'''
    if newline == True: print()
    response = input('Press "Enter" to quit: ')
    if response: sys.exit()

### Make Log
This function takes in an error object and its namespace to creates a log in the main directory.

In [ ]:
def makelog(obj,namespace,opnmsg="",handled=False):
    '''This function takes in an error object and its namespace to creates a log in the main directory.'''
    a = time.strftime("%Y/%m/%d %H:%M:%S",time.localtime(time.time()))
    b = "Unhandled" if not handled else "Handled"
    logitem = a + "," + namespace + ',' + b + "," + str(opnmsg) + "," + str(type(obj)) + "," + str(obj) + "\n"
    
    if 'log.txt' not in os.listdir():
        open('log.txt','wt',encoding='utf-8')
    with open('log.txt','at',encoding='utf-8') as fl:
        fl.write(logitem)

### Automate Import Data
This comprises the main program of this automation script, it shall run the functions declared above, do the job automatically and return successful/failure
messages once the program come to an end.

Procedure:
1. Check whether the given source importing folder is empty
2. Check whether a file is in XLS format, skip any file which is not and returning the count for how many non-XLS files exist in the given directory
3. Run function xml2csv to read all XLS files into a CSV formatted temp string
4. Write the temp CSV string into an actual temp CSV file
5. Parse the actual temp CSV into a pandas dataframe
6. Delete the actual temp CSV file
7. Run the functions in order:
    1. ins_import_date(dataframe)
    2. vehicle_process(dataframe)
    3. truck_type_process(dataframe,config)
    4. date_process(dataframe)
8. Write the dataframe into a workbook
9. Save the workbook to the destination excel file
10. Output importation results & benchmarking results

Extra Requirements:
* There should be a counter for how many files processed/imported (number, lines & bytes), and another for how many files to process in total
* This program should display a progress bar enabling the user to visualize the progress of the importation process
* This program shall print a message to inform the user on how many files are successfully imported, how many are skipped for some specified reason, total elapsed time for the entire importation process, the size of the content imported in bytes
* This program should have a benchmarking message (can be disabled) displaying all elapsed time taken in different stages of the entire program
* At the end of all source file importing, this program shall ask the user whether to move all source files to the "Data" folder beside the source file folder

Packages used:
* os
    * os.listdir()
    * os.strerror()
* sys
    * sys.exit()
* errno
    * errno.errorcode [Dict]
* openpyxl
    * openpyxl.Workbook
    * openpyxl.load_workbook
    * openpyxl.utils.dataframe.dataframe_to_rows()
* pandas (for data processing and dataframe formatting)
    * pandas.io.parsers.read_csv()

In [ ]:
def automate_import_data(config):
    global tmpcsv
    importsuccess = 0
    importempty = 0
    imported = []
    emptyfile = []
    nonxlscount = 0
    nonxls = []
    skippedforerrorcount = 0
    skippedforerror = []
    bytewriten = 0
    linewriten = 0

    try:
        filelist = os.listdir(config['importpath'])
        if filelist == ["desktop.ini"] or not filelist:
            raise DirEmptyException(config['importpath'])
        if "desktop.ini" in filelist: totalcount = len(filelist) - 1
        else: totalcount = len(filelist)
        print(f"Total {totalcount} files to be imported")
    except DirEmptyException as e:
        makelog(e.msg,automate_import_data.__name__,handled=True)
        print(e.msg)
        askquit()
    except Exception as e:
        em = f"Error accessing {config['importpath']}"
        makelog(e,automate_import_data.__name__,em)
    else:
        t0 = Etime()
        makelog("","START",f"Started Importing {totalcount} files",True)

        for file in filelist:
            if not file.lower().endswith(".xls"):
                if file.lower() == "desktop.ini":
                    continue
                nonxlscount += 1
                nonxls.append(file)
                continue
            else:
                try:
                    result = xml2csv(file,config)
                except UnicodeError:
                    result = xls2csv(file,config)
                except Exception as e:
                    em = f"Error reading data from '{file}'"
                    makelog(e,automate_import_data.__name__,em)
                finally:
                    if result == "success":
                        importsuccess += 1
                        imported.append(file)
                    elif result == "empty":
                        importempty += 1
                        emptyfile.append(file)
                    elif result == "error":
                        skippedforerrorcount += 1
                        skippedforerror.append(file)                        
                    if(importsuccess + importempty + skippedforerrorcount)% config['dispsplit1'] == 0: print("", importsuccess + importempty + skippedforerrorcount)
        else:
            if(importsuccess + importempty)% config['dispsplit1'] != 0: print("", importsuccess + importempty)
            if nonxlscount:
                    print("We noticed that there are file(s) not in .xls format in the import source directory!")

            with open(config['destpath'] + 'tmpcsv.csv','wt',encoding='utf-8') as fl:
                bytewriten += fl.write(tmpcsv)
                print(f"Total {bytewriten} bytes read from source files")
                tmpcsv = ""
                fl.close()

        if config['benchmarking']: t1 = Etime()

    try:
        df1 = pd.io.parsers.read_csv(config['destpath'] + 'tmpcsv.csv',index_col=False)
        os.remove(config['destpath'] + 'tmpcsv.csv')
        ins_import_date(df1)
        vehicle_process(df1)
        truck_type_process(df1,config)
        date_process(df1)

        if config['benchmarking']: t2 = Etime()

        wb = load_workbook(config['destpath'] + config['mainexcel'])
        ws = wb[config['mainexcelsheet']]

        if config['benchmarking']: t3 = Etime()

        for r in dataframe_to_rows(df1, index=False, header=False):
            ws.append(r)
            linewriten += 1
            if linewriten % 10 == 0: print("|",end='')
            if linewriten % config['dispsplit2'] == 0: print("",linewriten)
        print("",linewriten)

        wb.save(config['destpath'] + config['mainexcel'])
    except IOError as e:
        em = f"Error accessing {config['destpath']}"
        makelog(e,automate_import_data.__name__,em,True)
        print(os.strerror(e.errno))
        askquit()
    except Exception as e:
        em = f"Error writing data into '{config['mainexcel']}'"
        makelog(e,automate_import_data.__name__,em)
    else:
        t4 = Etime()
        makelog("","END",f"Ended Importing {importsuccess} files",True)

        print(f"Completed importing {importsuccess} out of {totalcount} file(s) in {t4.difftime(t0)}:")
        print(imported[0]+" ... ",end='')
        if len(imported) > 1: print(imported[-1],end='')
        print(f"\nTotal {linewriten} lines imported")
        print()
        if importempty:
            print(f"{importempty} file(s) are empty and skipped")
            print(emptyfile[0]+" ... ",end='')
            if len(emptyfile) > 1: print(emptyfile[-1],end='')
            print()
        if nonxlscount:
            print(f"{nonxlscount} non XLS file(s) detected and skipped:")
            [print(x,end=', ') for x in nonxls]
            print()
        if skippedforerrorcount:
            print(f"{skippedforerrorcount} file(s) with unknown error are skipped:")
            [print(x,end=', ') for x in skippedforerror]
            print()
        if config['benchmarking']:
          print(f"\nStage 1: {t1.difftime(t0)}",f"Stage 2: {t2.difftime(t1)}",\
                f"Stage 3: {t3.difftime(t2)}",f"Stage 4: {t4.difftime(t3)}\n")

        movefiles(imported,config)